In [ ]:
import pandas as ps
import os
import numpy as np
import globalConstants as gConst

In [ ]:
PARENT_FOLDER = gConst.PARENT_FOLDER
INPUT_CSV = PARENT_FOLDER + "trainingImageMeta.csv"
PSP_NET_BIG = PARENT_FOLDER + "output_120k.csv"
PSP_NET_SUPP = PARENT_FOLDER + "output_QC_substitute_images.csv"
SEATTLE_STRAIGHT_CSV = PARENT_FOLDER + "SeattleStraightCandidates.csv"
SEATTLE_SIDE_CSV = PARENT_FOLDER + "SeattleSideCandidates.csv"
CITY_STRAIGHT_CSV = PARENT_FOLDER + "CityStraightCandidates.csv"
CITY_SIDE_CSV = PARENT_FOLDER + "CitySideCandidates.csv"
SUBURB_STRAIGHT_CSV = PARENT_FOLDER + "SuburbStraightCandidates.csv"
SUBURB_SIDE_CSV = PARENT_FOLDER + "SuburbSideCandidates.csv"

In [ ]:
def loadDatasets():
    a = ps.read_csv(PSP_NET_BIG)
    b = ps.read_csv(PSP_NET_SUPP)
    PSP_NET = a.append(b)
    trainingImgMeta = ps.read_csv(INPUT_CSV)
    numImgs = len(trainingImgMeta['id'])
    joinVals = []
    for index in range(numImgs):
        joinVals.append(trainingImgMeta['id'][index] + "_" + str(trainingImgMeta['GSVHeading'][index]))
    trainingImgMeta['joinVal'] = joinVals
    combinedSet = trainingImgMeta.merge(PSP_NET, left_on='joinVal', right_on='panid',how="inner")
    return([combinedSet,trainingImgMeta])

In [ ]:
def createCategory(inData,categories,categoryName):
    tempData = np.zeros((len(inData['wall']),1))
    index=0
    for category in categories:
        print(category)
        tempData = np.add(tempData,np.array(inData[category]).reshape(len(inData['wall']),1))
    inData[categoryName] = tempData

In [ ]:
def createCategories(inData):
    # crate the built environment category
    builtEnv = ['wall','building','road','windowpane','sidewalk','hovel','house','fence','railing',
               'signboard','skyscraper','path','stairs','runway','screen door','stairway','bridge',
               'bench','booth','awning','streetlight','television receiver','pole','bannister','escalator',
               'fountain','swimming pool','step','sculpture','traffic light','pier']
    createCategory(inData,builtEnv,'built_env')

    build2 = ['house','building','skyscraper','windowpane','hovel']
    createCategory(inData,build2,'build2')
    
    # create the accessibility category 
    accessibility = ['sidewalk','escalator','path','stairs','stairway','bench','step']
    createCategory(inData,accessibility,'accessibility')
    
    # create the allNature category
    allNature = ['tree','grass','plant','field','land','flower','water','sea','waterfall','lake','earth',
                'mountain','rock','sky','sand','hill','dirt track']
    createCategory(inData,allNature,'all_nature')
    
    # create the greenspace cateogry 
    greenspace = ['tree','grass','plant','field','flower']
    createCategory(inData,greenspace,'greenspace')
    
    # create the bluespace category 
    bluespace = ['water','sea','waterfall','lake']
    createCategory(inData,bluespace,'bluespace')
    
    # create the otherNature category
    otherNature = ['earth','mountain','rock','sky','sand','hill','dirt track','land']
    createCategory(inData,otherNature,'other_nature')
    
    # create the animate category
    animate = ['person','boat','car','bus','truck','airplane','van','ship','minibike','animal','bicycle']
    createCategory(inData,animate,'animate')

In [ ]:
combinedSet,trainingImg = loadDatasets()
createCategories(combinedSet)

In [ ]:
def createSubsetDatasets(inData):
    SeattleSubset = inData[inData['proposed_p']==0]
    SeattleStraight = SeattleSubset[SeattleSubset['viewCat']=='straightAngle']
    SeattleStraight.to_csv(SEATTLE_STRAIGHT_CSV)
    SeattleSide = SeattleSubset[SeattleSubset['viewCat']=='sideAngle']
    SeattleSide.to_csv(SEATTLE_SIDE_CSV)
    NotSeattleSubset = inData[inData['proposed_p']>0]
    CitySubset = NotSeattleSubset[NotSeattleSubset['UATYP10']=='U']
    CityStraight = CitySubset[CitySubset['viewCat']=='straightAngle']
    CityStraight.to_csv(CITY_STRAIGHT_CSV)
    CitySide = CitySubset[CitySubset['viewCat']=='sideAngle']
    CitySide.to_csv(CITY_SIDE_CSV)
    SuburbSubset = NotSeattleSubset[NotSeattleSubset['UATYP10']=='C']
    SuburbStraight = SuburbSubset[SuburbSubset['viewCat']=='straightAngle']
    SuburbStraight.to_csv(SUBURB_STRAIGHT_CSV)
    SuburbSide = SuburbSubset[SuburbSubset['viewCat']=='sideAngle']
    SuburbSide.to_csv(SUBURB_SIDE_CSV)

In [ ]:
createSubsetDatasets(combinedSet)